In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "klue/bert-base" # korea embedding model
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

No sentence-transformers model found with name klue/bert-base. Creating a new one with mean pooling.


In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1, # seperator로만 청크를 나누게 된다.
    chunk_overlap=0
)

loader = TextLoader("../docs/travel.txt", encoding="utf-8")

documents = loader.load_and_split(text_splitter=text_splitter)
documents

Created a chunk of size 52, which is longer than the specified 1
Created a chunk of size 46, which is longer than the specified 1
Created a chunk of size 43, which is longer than the specified 1
Created a chunk of size 47, which is longer than the specified 1
Created a chunk of size 41, which is longer than the specified 1
Created a chunk of size 41, which is longer than the specified 1
Created a chunk of size 47, which is longer than the specified 1
Created a chunk of size 38, which is longer than the specified 1
Created a chunk of size 40, which is longer than the specified 1
Created a chunk of size 41, which is longer than the specified 1
Created a chunk of size 38, which is longer than the specified 1
Created a chunk of size 46, which is longer than the specified 1
Created a chunk of size 36, which is longer than the specified 1
Created a chunk of size 35, which is longer than the specified 1
Created a chunk of size 45, which is longer than the specified 1
Created a chunk of size 3

[Document(metadata={'source': '../docs/travel.txt'}, page_content='여행을 준비하는 방법은 무엇인가? 필요한 짐을 꾸리는 팁과 함께 여행의 기본 요소를 알아보자.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='여행 가방을 꾸리는 노하우. 최소한의 짐으로 최대한의 효율을 내는 팁을 공유합니다.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='효율적인 여행 계획 세우기. 예산을 절약하면서도 만족스러운 여행을 하는 방법.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='여행 경로를 설계하는 법. 최적의 동선을 계획하여 피로도를 줄이는 방법을 소개합니다.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='여행 일정 짜는 방법. 관광 명소를 효과적으로 방문하기 위한 스케줄링 팁.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='여행지에서의 안전한 이동 수단. 대중교통과 렌터카, 어떤 것이 유리할까?.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='여행 중 건강을 유지하는 법. 장거리 이동 시 피로를 줄이는 방법과 유용한 건강 팁.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='여행 중 의사소통 팁. 현지 언어를 몰라도 효과적으로 소통하는 방법.'),
 Document(metadata={'source': '../docs/travel.txt'}, page_content='여행 예산을 절약하

In [6]:
from langchain_chroma import Chroma

db = Chroma.from_documents(documents, hf_embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [7]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k": 20, "lambda_mult": 0.5}
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
message = """
Anser this question using the provided context only.
아래 제공된 내용만 참고하여 사용자의 질문에 답변해줘. 모르면 솔직하게 그냥 모른다고 답해.

Context:
{context}

User Import:
{input}
"""

propmt_templte = ChatPromptTemplate.from_messages([
    ("human", message)
])

In [9]:
from langchain import hub
propmt_templte = hub.pull("langchain-ai/retrieval-qa-chat")
propmt_templte

/Users/hong/.pyenv/versions/3.12.11/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [10]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [12]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = { 
    "context": retriever, # 사용자가 입력한 값을 토대로 검색된 내용을 넣음
    "input": RunnablePassthrough() # 사용자 입력값을 그대로 넣는 객체
} | propmt_templte | llm | parser

# context = retriever.invoke(question)
# input = RunnablePassthrough().invoke(question)

In [ ]:
rag_chain.invoke("이색적인 숙소에는 어떤 곳이 있을까?")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


'이색적인 숙소로는 트리하우스, 동굴 호텔, 빙하 호텔 등 독특한 숙소 경험을 제공하는 곳들이 있습니다.'